In [ ]:
import ipywidgets as widgets
import os
import pandas as pd
import ray
import subprocess
import tempfile

from IPython.display import display

ray.init(redis_address=os.environ["REDIS_ADDRESS"])

In [ ]:
class _EventRecursionContextManager(object):
    def __init__(self):
        self.should_recurse = True
   
    def __enter__(self):
        self.should_recurse = False
       
    def __exit__(self, *args):
        self.should_recurse = True

total_time_value = '% total time'
total_tasks_value = '% total tasks'
   
def get_sliders(update):
    start_box = widgets.FloatText(
        description='Start Time:',
        disabled=True,
    )
    end_box = widgets.FloatText(
        description='End Time:',
        disabled=True,
    )
    range_slider = widgets.IntRangeSlider(
        value=[70, 100],
        min=0,
        max=100,
        step=1,
        description='%:',
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.0i%',
    )
    num_tasks_box = widgets.IntText(
        description='Num Tasks:',
        disabled=False
    )
 
    breakdown_opt = widgets.Dropdown(
        options=[total_time_value, total_tasks_value],
        value=total_tasks_value,
        description="Selection Options:"
    )
   
    INIT_EVENT = 'INIT'
    out_recursion = _EventRecursionContextManager()
   
    def update_wrapper(event):
        if not out_recursion.should_recurse:
            return
       
        with out_recursion:
            smallest, largest, num_tasks = ray.global_state.job_length()
            diff = largest - smallest
            if num_tasks is not 0: 
                if event == INIT_EVENT:
                    if breakdown_opt.value == total_tasks_value:
                        num_tasks_box.value = -min(10000, num_tasks)
                        range_slider.value = (int(100 - (100. * -num_tasks_box.value) / num_tasks), 100)
                    else:
                        low, high = map(lambda x: x / 100., range_slider.value)
                        start_box.value = round(diff * low, 2)
                        end_box.value = round(diff * high, 2)
                elif event['owner'] == start_box:
                    if start_box.value > end_box.value:
                        start_box.value = end_box.value
                    elif start_box.value < 0:
                        start_box.value = 0

                    low, high = range_slider.value
                    range_slider.value = (int((start_box.value * 100.) / diff), high)
                elif event['owner'] == end_box:
                    if start_box.value > end_box.value:
                        end_box.value = start_box.value
                    elif end_box.value > diff:
                        end_box.value = diff

                    low, high = range_slider.value
                    range_slider.value = (low, int((end_box.value * 100.) / diff))
                elif event['owner'] == breakdown_opt:
                    if breakdown_opt.value == total_tasks_value:
                        start_box.disabled = True
                        end_box.disabled = True
                        num_tasks_box.disabled = False
                        num_tasks_box.value = min(10000, num_tasks)
                        range_slider.value = (int(100 - (100. * num_tasks_box.value) / num_tasks), 100)
                    else:
                        start_box.disabled = False
                        end_box.disabled = False
                        num_tasks_box.disabled = True
                        range_slider.value = (int((start_box.value * 100.) / diff),
                                              int((end_box.value * 100.) / diff))
                elif event['owner'] == range_slider:
                    low, high = map(lambda x: x / 100., range_slider.value)
                    if breakdown_opt.value == total_tasks_value:
                        old_low, old_high = event['old']
                        new_low, new_high = event['new']
                        if old_low != new_low:
                            range_slider.value = (new_low, 100)
                            num_tasks_box.value = -(100. - new_low) / 100. * num_tasks
                        else:
                            range_slider.value = (0, new_high)
                            num_tasks_box.value = new_high / 100. * num_tasks
                    else:
                        start_box.value = round(diff * low, 2)
                        end_box.value = round(diff * high, 2)
                elif event['owner'] == num_tasks_box:
                    if num_tasks_box.value > 0:
                        range_slider.value = (0, int(100 * float(num_tasks_box.value) / num_tasks))
                    elif num_tasks_box.value < 0:
                        range_slider.value = (100 + int(100 * float(num_tasks_box.value) / num_tasks), 100)
                else:
                    raise ValueError('Unknown event owner!')

                if not update:
                    return

                diff = largest - smallest
                low, high = map(lambda x: x / 100., range_slider.value)

                if breakdown_opt.value == total_time_value:
                    tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
                elif breakdown_opt.value == total_tasks_value:
                    if range_slider.value[0] == 0:
                        tasks = ray.global_state.task_profiles(num_slice=int(num_tasks * high), fwd=True)
                    else:
                        tasks = ray.global_state.task_profiles(num_slice=int(num_tasks * (high - low)), fwd=False)
                else:
                    raise ValueError('Value "{}" is not a legal breakdown value'.format(breakdown_opt.value))

                update(smallest, largest, num_tasks, tasks)
       
    range_slider.observe(update_wrapper, names='value')
    breakdown_opt.observe(update_wrapper, names='value')
    start_box.observe(update_wrapper, names='value')
    end_box.observe(update_wrapper, names='value')
    num_tasks_box.observe(update_wrapper, names='value')
   
    update_wrapper(INIT_EVENT)
   
    display(start_box, end_box, range_slider, num_tasks_box, breakdown_opt)
   
    return start_box, end_box, range_slider, breakdown_opt

In [ ]:
def task_timeline():
    path_input = widgets.Button(description="View task timeline")

    breakdown_basic = "Basic"
    breakdown_task = "Task Breakdowns"
    
    breakdown_opt = widgets.Dropdown(
        options=["Basic", "Task Breakdowns"],
        value="Basic",
        description="View options:",
        disabled=False,
    )
    
    start_box, end_box, range_slider, time_opt = get_sliders(False)
    display(breakdown_opt)
    display(path_input)

    def find_trace2html():
        trace2html = os.path.join("/tmp", "catapult", "tracing", "bin", "trace2html")
        if not os.path.exists(trace2html): 
            cmd = ["git", "clone", "https://github.com/catapult-project/catapult.git", "/tmp/catapult"]
            subprocess.check_output(cmd) 
            print("Cloning catapult to /tmp.")
        assert os.path.exists(trace2html)
        return trace2html

    def handle_submit(sender):
        tmp = tempfile.mktemp() + ".json"
        tmp2 = tempfile.mktemp() + ".html"
        
        if breakdown_opt.value == breakdown_basic:
            breakdown = False
        elif breakdown_opt.value == breakdown_task:
            breakdown = True
        else:
            raise ValueError('Unexpected breakdown value "{}"'.format(breakdown_opt.value))
            
        low, high = map(lambda x: x / 100., range_slider.value)
        
        smallest, largest, num_tasks = ray.global_state._job_length()
        diff = largest - smallest

        if time_opt.value == total_time_value:
            tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
        elif time_opt.value == total_tasks_value:
            if range_slider.value[0] == 0:
                tasks = ray.global_state.task_profiles(num_slice=int(num_tasks * high), fwd=True)
            else:
                tasks = ray.global_state.task_profiles(num_slice=int(num_tasks * (high - low)), fwd=False)
        else:
            raise ValueError('Unexpected time value "{}"'.format(time_opt.value))
        
        print('{} tasks to trace'.format(len(tasks)))
        print("Dumping task profiling data to " + tmp)
        ray.global_state.dump_catapult_trace(tmp, tasks, breakdowns=breakdown)
        print("Converting chrome trace to " + tmp2)
        trace2html = find_trace2html()
        subprocess.check_output(["python2", trace2html, tmp, '--output', tmp2])
        print("Opening html file in browser...")
        subprocess.Popen(["open", "-a", "Google Chrome", tmp2])

    path_input.on_click(handle_submit)

task_timeline()